# NLP and DeepLearning Assignment1
### Working progress

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import sklearn
import random

### Define the softmax function
#### q1_softmax.py

In [2]:
def softmax(x):
    """Compute the softmax function for each row of the input x.

    It is crucial that this function is optimized for speed because
    it will be used frequently in later code. You might find numpy
    functions np.exp, np.sum, np.reshape, np.max, and numpy
    broadcasting useful for this task.

    Numpy broadcasting documentation:
    http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

    You should also make sure that your code works for a single
    N-dimensional vector (treat the vector as a single row) and
    for M x N matrices. This may be useful for testing later. Also,
    make sure that the dimensions of the output match the input.

    You must implement the optimization in problem 1(a) of the
    written assignment!

    Arguments:
    x -- A N dimensional vector or M x N dimensional numpy matrix.

    Return:
    x -- You are allowed to modify x in-place
    """
    orig_shape = x.shape

    if len(x.shape) > 1:
        # Matrix
        ### YOUR CODE HERE
        # raise NotImplementedError

        maxs = np.max(x, axis=1) # An array of max values of each line
        x = x - maxs.reshape(maxs.shape[0], 1) # for each line, subtract the max value
        sums = np.sum(np.exp(x), axis = 1)
        x = np.exp(x) / sums.reshape(sums.shape[0], 1)
        ### END YOUR CODE
    else:
        # Vector
        ### YOUR CODE HERE
        # raise NotImplementedError
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

        ### END YOUR CODE

    assert x.shape == orig_shape
    return x

### Test the numpy array broadcasting operations

In [3]:
x = np.array([
    [0.26894142, 0.73105858, 23, 1],
    [0.26894142, 0.73105858, 12, 2],
    [2, 3, 4, 5]])
maxs = np.max(x, axis=1) # An array of max values of each line
x = x - maxs.reshape(maxs.shape[0], 1)
x

array([[-22.73105858, -22.26894142,   0.        , -22.        ],
       [-11.73105858, -11.26894142,   0.        , -10.        ],
       [ -3.        ,  -2.        ,  -1.        ,   0.        ]])

In [4]:
sums = np.sum(np.exp(x), axis = 1)
x = np.exp(x) / sums.reshape(sums.shape[0], 1)
x

array([[  1.34284749e-10,   2.13167810e-10,   9.99999999e-01,
          2.78946809e-10],
       [  8.03965182e-06,   1.27623948e-05,   9.99933801e-01,
          4.53969243e-05],
       [  3.20586033e-02,   8.71443187e-02,   2.36882818e-01,
          6.43914260e-01]])

### Test the softmax function

In [5]:
def test_softmax_basic():
    """
    Some simple tests to get you started.
    Warning: these are not exhaustive.
    """
    print "Running basic tests..."
    test1 = softmax(np.array([1,2]))
    print test1
    ans1 = np.array([0.26894142,  0.73105858])
    assert np.allclose(test1, ans1, rtol=1e-05, atol=1e-06)

    test2 = softmax(np.array([[1001,1002],[3,4]]))
    print test2
    ans2 = np.array([
        [0.26894142, 0.73105858],
        [0.26894142, 0.73105858]])
    assert np.allclose(test2, ans2, rtol=1e-05, atol=1e-06)

    test3 = softmax(np.array([[-1001,-1002]]))
    print test3
    ans3 = np.array([0.73105858, 0.26894142])
    assert np.allclose(test3, ans3, rtol=1e-05, atol=1e-06)

    print "You should be able to verify these results by hand!\n"

In [6]:
test_softmax_basic()

Running basic tests...
[ 0.26894142  0.73105858]
[[ 0.26894142  0.73105858]
 [ 0.26894142  0.73105858]]
[[ 0.73105858  0.26894142]]
You should be able to verify these results by hand!



### Define the sigmoid and its gradient
#### q2_sigmoid.py

In [7]:
def sigmoid(x):
    """
    Compute the sigmoid function for the input here.

    Arguments:
    x -- A scalar or numpy array.

    Return:
    s -- sigmoid(x)
    """

    ### YOUR CODE HERE
    #raise NotImplementedError
    s = 1/ (1 + np.exp(-x))
    
    ### END YOUR CODE

    return s


def sigmoid_grad(s):
    """
    Compute the gradient for the sigmoid function here. Note that
    for this implementation, the input s should be the sigmoid
    function value of your original input x.

    Arguments:
    s -- A scalar or numpy array.

    Return:
    ds -- Your computed gradient.
    """

    ### YOUR CODE HERE
    #raise NotImplementedError
    
    ds = s * (1 - s)
    ### END YOUR CODE

    return ds

In [8]:
x = np.array([[1, 2], [-1, -2]])
sigmoid(x)

array([[ 0.73105858,  0.88079708],
       [ 0.26894142,  0.11920292]])

In [9]:
def test_sigmoid_basic():
    """
    Some simple tests to get you started.
    Warning: these are not exhaustive.
    """
    print "Running basic tests..."
    x = np.array([[1, 2], [-1, -2]])
    f = sigmoid(x)
    g = sigmoid_grad(f)
    print f
    f_ans = np.array([
        [0.73105858, 0.88079708],
        [0.26894142, 0.11920292]])
    assert np.allclose(f, f_ans, rtol=1e-05, atol=1e-06)
    print g
    g_ans = np.array([
        [0.19661193, 0.10499359],
        [0.19661193, 0.10499359]])
    assert np.allclose(g, g_ans, rtol=1e-05, atol=1e-06)
    print "You should verify these results by hand!\n"

In [10]:
test_sigmoid_basic()

Running basic tests...
[[ 0.73105858  0.88079708]
 [ 0.26894142  0.11920292]]
[[ 0.19661193  0.10499359]
 [ 0.19661193  0.10499359]]
You should verify these results by hand!



### Implement a gradient checker
#### q2_gradcheck.py

In [11]:
# First implement a gradient checker by filling in the following functions
def gradcheck_naive(f, x):
    """ Gradient check for a function f.

    Arguments:
    f -- a function that takes a single argument and outputs the
         cost and its gradients
    x -- the point (numpy array) to check the gradient at
    """

    rndstate = random.getstate()
    random.setstate(rndstate)
    fx, grad = f(x) # Evaluate function value at original point
    h = 1e-4        # Do not change this!

    # Iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    count = 0
    while not it.finished:
        ix = it.multi_index

        # Try modifying x[ix] with h defined above to compute
        # numerical gradients. Make sure you call random.setstate(rndstate)
        # before calling f(x) each time. This will make it possible
        # to test cost functions with built in randomness later.

        ### YOUR CODE HERE:
        #raise NotImplementedError
        random.setstate(rndstate)
        current_value = x[ix] 
        ### calculate outcomes when the input at ix point change -h
        x[ix] = current_value - h
        y1, _ = f(x)
        random.setstate(rndstate)
        ### calculate outcomes when the input at ix point change h
        x[ix] = current_value + h
        y2, _ = f(x)
        numgrad = (y2 - y1) / (2*h)
        ### END YOUR CODE

        # Compare gradients
        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))

        if reldiff > 1e-5:
            print "Gradient check failed."
            print "First gradient error found at index %s" % str(ix)
            print "Your gradient: %f \t Numerical gradient: %f" % (
                grad[ix], numgrad)
            count = count + 1

        it.iternext() # Step to next dimension

    print "Gradient check found: " + str(count) + " erors!"


def sanity_check():
    """
    Some basic sanity checks.
    """
    quad = lambda x: (np.sum(x ** 2), x * 2)

    print "Running sanity checks..."
    gradcheck_naive(quad, np.array(123.456))      # scalar test
    gradcheck_naive(quad, np.random.randn(3,))    # 1-D test
    gradcheck_naive(quad, np.random.randn(4,5))   # 2-D test
    print ""


In [12]:
sanity_check()

Running sanity checks...
Gradient check found: 0 erors!
Gradient check found: 0 erors!
Gradient check found: 0 erors!



### Implement one-layer NN forward and backward propagation
#### q2_neural.py

In [13]:
def forward_backward_prop(data, labels, params, dimensions):
    """
    Forward and backward propagation for a two-layer sigmoidal network

    Compute the forward propagation and for the cross entropy cost,
    and backward propagation for the gradients for all parameters.

    Arguments:
    data -- M x Dx matrix, where each row is a training example.
    labels -- M x Dy matrix, where each row is a one-hot vector.
    params -- Model parameters, these are unpacked for you.
    dimensions -- A tuple of input dimension, number of hidden units
                  and output dimension
    """

    ### Unpack network parameters (do not modify)
    ofs = 0
    Dx, H, Dy = (dimensions[0], dimensions[1], dimensions[2])

    W1 = np.reshape(params[ofs:ofs+ Dx * H], (Dx, H))
    ofs += Dx * H
    b1 = np.reshape(params[ofs:ofs + H], (1, H))
    ofs += H
    W2 = np.reshape(params[ofs:ofs + H * Dy], (H, Dy))
    ofs += H * Dy
    b2 = np.reshape(params[ofs:ofs + Dy], (1, Dy))

    ### YOUR CODE HERE: forward propagation
    #raise NotImplementedError
    
    N = dimensions[0]
    
    z1 = data.dot(W1) + b1
    h = sigmoid(z1)
    z2 = h.dot(W2) + b2
    y = softmax(z2)
    cost = (np.sum(-labels * np.log(y))) / N
    ### END YOUR CODE

    ### YOUR CODE HERE: backward propagation
    #raise NotImplementedError
    
    delta1 = (y - labels) / N
    gradb2 = np.sum(delta1, axis = 0)
    gradW2_ind = delta1[:, np.newaxis, :] * h[:,:,np.newaxis]
    gradW2 = np.sum(gradW2_ind, axis = 0)
    deriv_h = sigmoid_grad(h)
    delta2 = np.dot(delta1, W2.T)
    gradb1_ind = delta2 * deriv_h
    gradb1 = np.sum(gradb1_ind, axis = 0)
    #gradW1 = data.T.dot(delta2 * deriv_h)
    gradW1_ind = gradb1_ind[:, np.newaxis, :] * data[:, :, np.newaxis]
    gradW1 = np.sum(gradW1_ind, axis = 0)
    ### END YOUR CODE

    ### Stack gradients (do not modify)
    grad = np.concatenate((gradW1.flatten(), gradb1.flatten(),
        gradW2.flatten(), gradb2.flatten()))

    return cost, grad


In [14]:
def sanity_check():
    """
    Set up fake data and parameters for the neural network, and test using
    gradcheck.
    """
    print "Running sanity check..."

    N = 20
    dimensions = [10, 5, 10]
    data = np.random.randn(N, dimensions[0])   # each row will be a datum
    labels = np.zeros((N, dimensions[2]))
    for i in xrange(N):
        labels[i, random.randint(0,dimensions[2]-1)] = 1

    params = np.random.randn((dimensions[0] + 1) * dimensions[1] + (
        dimensions[1] + 1) * dimensions[2], )

    gradcheck_naive(lambda params:
        forward_backward_prop(data, labels, params, dimensions), params)
sanity_check()

Running sanity check...
Gradient check failed.
First gradient error found at index (11,)
Your gradient: -0.053841 	 Numerical gradient: -0.053851
Gradient check failed.
First gradient error found at index (24,)
Your gradient: 0.054640 	 Numerical gradient: 0.054652
Gradient check failed.
First gradient error found at index (49,)
Your gradient: 0.039215 	 Numerical gradient: 0.039203
Gradient check failed.
First gradient error found at index (61,)
Your gradient: 0.033504 	 Numerical gradient: 0.033515
Gradient check failed.
First gradient error found at index (62,)
Your gradient: 0.033225 	 Numerical gradient: 0.033206
Gradient check failed.
First gradient error found at index (63,)
Your gradient: -0.175517 	 Numerical gradient: -0.175534
Gradient check failed.
First gradient error found at index (75,)
Your gradient: -0.019533 	 Numerical gradient: -0.019548
Gradient check failed.
First gradient error found at index (77,)
Your gradient: 0.086491 	 Numerical gradient: 0.086501
Gradient c

### Step by step implementation and tests

In [15]:
N = 20
dimensions = [10, 5, 10]
data = np.random.randn(N, dimensions[0])   # each row will be a datum
labels = np.zeros((N, dimensions[2]))
ofs = 0
Dx, H, Dy = (dimensions[0], dimensions[1], dimensions[2])

params = np.random.randn((dimensions[0] + 1) * dimensions[1] + (
        dimensions[1] + 1) * dimensions[2], )

W1 = np.reshape(params[ofs:ofs+ Dx * H], (Dx, H))
ofs += Dx * H
b1 = np.reshape(params[ofs:ofs + H], (1, H))
ofs += H
W2 = np.reshape(params[ofs:ofs + H * Dy], (H, Dy))
ofs += H * Dy
b2 = np.reshape(params[ofs:ofs + Dy], (1, Dy))

for i in xrange(N):
    labels[i, random.randint(0,dimensions[2]-1)] = 1

In [16]:
z1 = data.dot(W1) + b1
h = sigmoid(z1)
z2 = h.dot(W2) + b2
y = softmax(z2)
cost = np.sum(-labels * np.log(y))
cost

56.20112769097463

# word2Vec

In [29]:
def normalizeRows(x):
    """ Row normalization function

    Implement a function that normalizes each row of a matrix to have
    unit length.
    """

    ### YOUR CODE HERE
    #raise NotImplementedError
    norm = np.sqrt(np.sum(x*x, axis = 1))
    x = x / norm.reshape(norm.shape[0], 1)
    ### END YOUR CODE

    return x

In [34]:
x = np.array([[2,3,4], [4,5,6]])
y = normalizeRows(x)
np.sum(y[0] *y[0])

1.0

In [111]:
def test_normalize_rows():
    print "Testing normalizeRows..."
    x = normalizeRows(np.array([[3.0,4.0],[1, 2]]))
    print x
    ans = np.array([[0.6,0.8],[0.4472136,0.89442719]])
    assert np.allclose(x, ans, rtol=1e-05, atol=1e-06)
    print ""

test_normalize_rows()

Testing normalizeRows...
[[ 0.6         0.8       ]
 [ 0.4472136   0.89442719]]



In [287]:
def softmaxCostAndGradient(predicted, target, outputVectors, dataset):
    """ Softmax cost function for word2vec models

    Implement the cost and gradients for one predicted word vector
    and one target word vector as a building block for word2vec
    models, assuming the softmax prediction function and cross
    entropy loss.

    Arguments:
    predicted -- numpy ndarray, predicted word vector (\hat{v} in
                 the written component)
    target -- integer, the index of the target word
    outputVectors -- "output" vectors (as rows) for all tokens
    dataset -- needed for negative sampling, unused here.

    Return:
    cost -- cross entropy cost for the softmax word prediction
    gradPred -- the gradient with respect to the predicted word
           vector
    grad -- the gradient with respect to all the other word
           vectors (should be: the gradient with respect to the output vectors)

    We will not provide starter code for this function, but feel
    free to reference the code you previously wrote for this
    assignment!
    """
    
    ### YOUR CODE HERE
    
    #raise NotImplementedError
    
    ## Forward cost calculation
    
    #"""
    
    #print "my code: softmaxCostAndGradient"
    
    y = np.zeros(outputVectors.shape[0])
    y[target] = 1
    y_hat = softmax(np.dot(predicted, outputVectors.T))
    cost = np.sum(-y * np.log(y_hat))
    
    ## Gradient calculation
    y_diff = y_hat - y
    gradPred = np.dot(y_diff, outputVectors)
    grad = np.outer(y_diff, predicted)
    
    """
    
    print "reference code: softmaxCostAndGradient"

    affine = np.dot(outputVectors, predicted)
    softmaxOutput = softmax(affine)[:, np.newaxis]
    cost  = -np.log(softmaxOutput[target])

    gradPred = -outputVectors[target] +  \
    np.sum(outputVectors*softmaxOutput, axis=0).transpose()
    grad = np.dot(softmaxOutput, predicted[:, np.newaxis].transpose())
    grad[target,:] -= predicted
    
    """
    
    ### END YOUR CODE

    return cost, gradPred, grad


In [288]:
def getNegativeSamples(target, dataset, K):
    """ Samples K indexes which are not the target """

    indices = [None] * K
    for k in xrange(K):
        newidx = dataset.sampleTokenIdx()
        while newidx == target:
            newidx = dataset.sampleTokenIdx()
        indices[k] = newidx
    return indices

In [289]:
def negSamplingCostAndGradient(predicted, target, outputVectors, dataset,
                               K=10):
    """ Negative sampling cost function for word2vec models

    Implement the cost and gradients for one predicted word vector
    and one target word vector as a building block for word2vec
    models, using the negative sampling technique. K is the sample
    size.

    Note: See test_word2vec below for dataset's initialization.

    Arguments/Return Specifications: same as softmaxCostAndGradient
    """

    # Sampling of indices is done for you. Do not modify this if you
    # wish to match the autograder and receive points!
    indices = [target]
    indices.extend(getNegativeSamples(target, dataset, K))
    
    ### YOUR CODE HERE
    #raise NotImplementedError
    
    ## Forward cost calculation
    
    #"""
    
    #print "my code: negSamplingCostAndGradient"
    
    # get the K+1 values from target and negative samples
    y1 = predicted.dot(outputVectors[indices, :].T)
    signs = -1 * np.ones(len(indices))
    signs[0] = 1
    y2 = sigmoid(y1 * signs)
    cost = -1 * np.sum(np.log(y2))
    
    ## Gradient calculation
    y3 = (y2 - 1) * signs
    gradPred = np.sum(y3[:, np.newaxis] * outputVectors[indices, :], axis = 0)
    
    grad = np.zeros(outputVectors.shape)
    #grad[target] = (sigmoid(predicted.dot(outputVectors[target])) - 1) * predicted
    
    otherGrads = np.outer(y3, predicted)
    for i in range(len(indices)):
        grad[indices[i]] += otherGrads[i]
    
    """
    print "reference code: negSamplingCostAndGradient"
    
    negativeSamples = [dataset.sampleTokenIdx() for i in range(K)]
    sigmoidTargetPred = sigmoid(outputVectors[target,:].transpose().dot(predicted))
    cost = -np.log(sigmoidTargetPred)   
    gradPred = (sigmoidTargetPred - 1.0)*outputVectors[target,:]
    grad = np.zeros(outputVectors.shape)
    grad[target,:] = predicted * (sigmoidTargetPred - 1.0)

    for sample in negativeSamples:
        sigmoidSamplePredicted = \
        sigmoid(-outputVectors[sample,:].transpose().dot(predicted))
        cost -= np.log(sigmoidSamplePredicted)
        gradPred += (1.0 - sigmoidSamplePredicted)*outputVectors[sample,:]
        grad[sample,:] += (1.0 - sigmoidSamplePredicted)*predicted.transpose()
    
    
    """
    
    ### END YOUR CODE

    return cost, gradPred, grad

In [290]:
def testCostAndGradient(target, predictedAndOutputVectors, dataset,
                       costAndGradient=softmaxCostAndGradient):
    
    allGrad = np.zeros(predictedAndOutputVectors.shape)
    
    predicted = predictedAndOutputVectors[0]
    outputVectors = predictedAndOutputVectors[1:]
    
    cost, gradPred, grad = costAndGradient(predicted, target, outputVectors,
                                          dataset)
    allGrad[0] = gradPred
    allGrad[1:] = grad
    
    return cost, allGrad

In [291]:
dataset = type('dummy', (), {})()
def dummySampleTokenIdx():
    return random.randint(0, 4)

def getRandomContext(C):
    tokens = ["a", "b", "c", "d", "e"]
    return tokens[random.randint(0,4)], \
        [tokens[random.randint(0,4)] for i in xrange(2*C)]
dataset.sampleTokenIdx = dummySampleTokenIdx
dataset.getRandomContext = getRandomContext

In [292]:
dummy_vectors = normalizeRows(np.random.randn(6,3))
dummy_vectors

array([[-0.36597665,  0.18481607, -0.91208778],
       [ 0.64868529, -0.5850643 , -0.4867311 ],
       [ 0.16416583, -0.0599129 ,  0.98461161],
       [-0.01344871,  0.80812007, -0.58886423],
       [ 0.9874962 , -0.04726513,  0.15039038],
       [ 0.47581947,  0.80895638,  0.34523239]])

In [293]:
print "==== Gradient check for costAndGradient ===="
gradcheck_naive(lambda vec: testCostAndGradient(3, 
                                                vec,
                                                dataset, 
                                                negSamplingCostAndGradient), 
                dummy_vectors)

==== Gradient check for costAndGradient ====
Gradient check failed.
First gradient error found at index (0, 1)
Your gradient: 0.165732 	 Numerical gradient: 0.165701
Gradient check failed.
First gradient error found at index (1, 0)
Your gradient: -0.767939 	 Numerical gradient: -0.767714
Gradient check failed.
First gradient error found at index (1, 1)
Your gradient: 0.387805 	 Numerical gradient: 0.388000
Gradient check failed.
First gradient error found at index (1, 2)
Your gradient: -1.913860 	 Numerical gradient: -1.913596
Gradient check failed.
First gradient error found at index (2, 0)
Your gradient: -0.301973 	 Numerical gradient: -0.301915
Gradient check failed.
First gradient error found at index (2, 1)
Your gradient: 0.152495 	 Numerical gradient: 0.152585
Gradient check failed.
First gradient error found at index (2, 2)
Your gradient: -0.752579 	 Numerical gradient: -0.752546
Gradient check failed.
First gradient error found at index (3, 0)
Your gradient: -0.487680 	 Numeric

In [294]:
def skipgram(currentWord, C, contextWords, tokens, inputVectors, outputVectors,
             dataset, word2vecCostAndGradient=softmaxCostAndGradient):
    """ Skip-gram model in word2vec

    Implement the skip-gram model in this function.

    Arguments:
    currrentWord -- a string of the current center word
    C -- integer, context size
    contextWords -- list of no more than 2*C strings, the context words
    tokens -- a dictionary that maps words to their indices in
              the word vector list
    inputVectors -- "input" word vectors (as rows) for all tokens
    outputVectors -- "output" word vectors (as rows) for all tokens
    word2vecCostAndGradient -- the cost and gradient function for
                               a prediction vector given the target
                               word vectors, could be one of the two
                               cost functions you implemented above.

    Return:
    cost -- the cost function value for the skip-gram model
    grad -- the gradient with respect to the word vectors
    """

    cost = 0.0
    gradIn = np.zeros(inputVectors.shape)
    gradOut = np.zeros(outputVectors.shape)

    ### YOUR CODE HERE
    #raise NotImplementedError
    
    predicted_ind = tokens[currentWord]
    predicted = inputVectors[predicted_ind]
    
    for w in contextWords:
        target_ind = tokens[w]
        #target = outputVectors[target_ind]
        cost_target, grad_pred, grad_U = \
        word2vecCostAndGradient(predicted, target_ind, outputVectors, dataset)
        cost += cost_target
        gradIn[predicted_ind] += grad_pred
        gradOut += grad_U
        
    ### END YOUR CODE

    return cost, gradIn, gradOut


In [295]:
def cbow(currentWord, C, contextWords, tokens, inputVectors, outputVectors,
         dataset, word2vecCostAndGradient=softmaxCostAndGradient):
    """CBOW model in word2vec

    Implement the continuous bag-of-words model in this function.

    Arguments/Return specifications: same as the skip-gram model

    Extra credit: Implementing CBOW is optional, but the gradient
    derivations are not. If you decide not to implement CBOW, remove
    the NotImplementedError.
    """

    cost = 0.0
    gradIn = np.zeros(inputVectors.shape)
    gradOut = np.zeros(outputVectors.shape)

    ### YOUR CODE HERE
    #raise NotImplementedError
    target_ind = tokens[currentWord]
    target = outputVectors[target_ind]
    
    context_indices = []
    v_hat = np.zeros(inputVectors.shape[1])
    for w in contextWords:
        context_ind = tokens[w]
        context_indices.append(context_ind)
        v_hat += inputVectors[context_ind]
        
    cost_target, grad_pred, grad_U = \
    word2vecCostAndGradient(v_hat, target_ind, outputVectors, dataset)
    cost += cost_target
    gradOut += grad_U
    for i in range(len(context_indices)):
        gradIn[context_indices[i]] += grad_pred
        
    ### END YOUR CODE

    return cost, gradIn, gradOut


In [296]:
#############################################
# Testing functions below. DO NOT MODIFY!   #
#############################################

def word2vec_sgd_wrapper(word2vecModel, tokens, wordVectors, dataset, C,
                         word2vecCostAndGradient=softmaxCostAndGradient):
    batchsize = 50
    cost = 0.0
    grad = np.zeros(wordVectors.shape)
    N = wordVectors.shape[0]
    inputVectors = wordVectors[:N/2,:]
    outputVectors = wordVectors[N/2:,:]
    for i in xrange(batchsize):
        C1 = random.randint(1,C)
        centerword, context = dataset.getRandomContext(C1)

        if word2vecModel == skipgram:
            denom = 1
        else:
            denom = 1

        c, gin, gout = word2vecModel(
            centerword, C1, context, tokens, inputVectors, outputVectors,
            dataset, word2vecCostAndGradient)
        cost += c / batchsize / denom
        grad[:N/2, :] += gin / batchsize / denom
        grad[N/2:, :] += gout / batchsize / denom

    return cost, grad


def test_word2vec():
    """ Interface to the dataset for negative sampling """
    dataset = type('dummy', (), {})()
    def dummySampleTokenIdx():
        return random.randint(0, 4)

    def getRandomContext(C):
        tokens = ["a", "b", "c", "d", "e"]
        return tokens[random.randint(0,4)], \
            [tokens[random.randint(0,4)] for i in xrange(2*C)]
    dataset.sampleTokenIdx = dummySampleTokenIdx
    dataset.getRandomContext = getRandomContext

    random.seed(31415)
    np.random.seed(9265)
    dummy_vectors = normalizeRows(np.random.randn(10,3))
    dummy_tokens = dict([("a",0), ("b",1), ("c",2),("d",3),("e",4)])
    print "==== Gradient check for skip-gram ===="
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(
        skipgram, dummy_tokens, vec, dataset, 5, softmaxCostAndGradient),
        dummy_vectors)
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(
        skipgram, dummy_tokens, vec, dataset, 5, negSamplingCostAndGradient),
        dummy_vectors)
    print "\n==== Gradient check for CBOW      ===="
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(
        cbow, dummy_tokens, vec, dataset, 5, softmaxCostAndGradient),
        dummy_vectors)
    gradcheck_naive(lambda vec: word2vec_sgd_wrapper(
        cbow, dummy_tokens, vec, dataset, 5, negSamplingCostAndGradient),
        dummy_vectors)

    print "\n=== Results ==="
    print skipgram("c", 3, ["a", "b", "e", "d", "b", "c"],
        dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset)
    print skipgram("c", 1, ["a", "b"],
        dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset,
        negSamplingCostAndGradient)
    print cbow("a", 2, ["a", "b", "c", "a"],
        dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset)
    print cbow("a", 2, ["a", "b", "a", "c"],
        dummy_tokens, dummy_vectors[:5,:], dummy_vectors[5:,:], dataset,
        negSamplingCostAndGradient)

In [299]:
test_normalize_rows()
test_word2vec()

Testing normalizeRows...
[[ 0.6         0.8       ]
 [ 0.4472136   0.89442719]]

==== Gradient check for skip-gram ====
Gradient check failed.
First gradient error found at index (0, 2)
Your gradient: 0.148685 	 Numerical gradient: 0.148663
Gradient check failed.
First gradient error found at index (1, 1)
Your gradient: -0.109647 	 Numerical gradient: -0.109635
Gradient check failed.
First gradient error found at index (1, 2)
Your gradient: -0.099611 	 Numerical gradient: -0.099634
Gradient check failed.
First gradient error found at index (2, 2)
Your gradient: 0.529433 	 Numerical gradient: 0.529403
Gradient check failed.
First gradient error found at index (3, 2)
Your gradient: -0.026513 	 Numerical gradient: -0.026534
Gradient check failed.
First gradient error found at index (4, 2)
Your gradient: 0.052118 	 Numerical gradient: 0.052090
Gradient check failed.
First gradient error found at index (5, 1)
Your gradient: 0.180330 	 Numerical gradient: 0.180361
Gradient check failed.
Firs

First gradient error found at index (0, 2)
Your gradient: -0.028432 	 Numerical gradient: -0.028518
Gradient check failed.
First gradient error found at index (1, 0)
Your gradient: -3.498695 	 Numerical gradient: -3.498596
Gradient check failed.
First gradient error found at index (1, 1)
Your gradient: -1.232545 	 Numerical gradient: -1.232354
Gradient check failed.
First gradient error found at index (1, 2)
Your gradient: -0.913349 	 Numerical gradient: -0.913487
Gradient check failed.
First gradient error found at index (2, 0)
Your gradient: -3.544427 	 Numerical gradient: -3.544256
Gradient check failed.
First gradient error found at index (2, 1)
Your gradient: -1.112075 	 Numerical gradient: -1.111818
Gradient check failed.
First gradient error found at index (3, 0)
Your gradient: -3.639383 	 Numerical gradient: -3.639114
Gradient check failed.
First gradient error found at index (3, 1)
Your gradient: -1.313386 	 Numerical gradient: -1.312985
Gradient check failed.
First gradient e

In [303]:
# Save parameters every a few SGD iterations as fail-safe
SAVE_PARAMS_EVERY = 5000

import glob
import random
import numpy as np
import os.path as op
import cPickle as pickle


def load_saved_params():
    """
    A helper function that loads previously saved parameters and resets
    iteration start.
    """
    st = 0
    for f in glob.glob("saved_params_*.npy"):
        iter = int(op.splitext(op.basename(f))[0].split("_")[2])
        if (iter > st):
            st = iter

    if st > 0:
        with open("saved_params_%d.npy" % st, "r") as f:
            params = pickle.load(f)
            state = pickle.load(f)
        return st, params, state
    else:
        return st, None, None


def save_params(iter, params):
    with open("saved_params_%d.npy" % iter, "w") as f:
        pickle.dump(params, f)
        pickle.dump(random.getstate(), f)


def sgd(f, x0, step, iterations, postprocessing=None, useSaved=False,
        PRINT_EVERY=10):
    """ Stochastic Gradient Descent

    Implement the stochastic gradient descent method in this function.

    Arguments:
    f -- the function to optimize, it should take a single
         argument and yield two outputs, a cost and the gradient
         with respect to the arguments
    x0 -- the initial point to start SGD from
    step -- the step size for SGD
    iterations -- total iterations to run SGD for
    postprocessing -- postprocessing function for the parameters
                      if necessary. In the case of word2vec we will need to
                      normalize the word vectors to have unit length.
    PRINT_EVERY -- specifies how many iterations to output loss

    Return:
    x -- the parameter value after SGD finishes
    """

    # Anneal learning rate every several iterations
    ANNEAL_EVERY = 20000

    if useSaved:
        start_iter, oldx, state = load_saved_params()
        if start_iter > 0:
            x0 = oldx
            step *= 0.5 ** (start_iter / ANNEAL_EVERY)

        if state:
            random.setstate(state)
    else:
        start_iter = 0

    x = x0

    if not postprocessing:
        postprocessing = lambda x: x

    expcost = None

    for iter in xrange(start_iter + 1, iterations + 1):
        # Don't forget to apply the postprocessing after every iteration!
        # You might want to print the progress every few iterations.

        cost = None
        ### YOUR CODE HERE
        #raise NotImplementedError
        
        cost, grad = f(x)
        x = x - step*grad
        if postprocessing is not None:
            x = postprocessing(x)
        
        ### END YOUR CODE

        if iter % PRINT_EVERY == 0:
            if not expcost:
                expcost = cost
            else:
                expcost = .95 * expcost + .05 * cost
            print "iter %d: %f" % (iter, expcost)

        if iter % SAVE_PARAMS_EVERY == 0 and useSaved:
            save_params(iter, x)

        if iter % ANNEAL_EVERY == 0:
            step *= 0.5

    return x

In [305]:
def sgd_sanity_check():
    quad = lambda x: (np.sum(x ** 2), x * 2)

    print "Running sanity checks..."
    t1 = sgd(quad, 0.5, 0.01, 1000, PRINT_EVERY=100)
    print "test 1 result:", t1
    assert abs(t1) <= 1e-6

    t2 = sgd(quad, 0.0, 0.01, 1000, PRINT_EVERY=100)
    print "test 2 result:", t2
    assert abs(t2) <= 1e-6

    t3 = sgd(quad, -1.5, 0.01, 1000, PRINT_EVERY=100)
    print "test 3 result:", t3
    assert abs(t3) <= 1e-6

    print ""
sgd_sanity_check()

Running sanity checks...
iter 100: 0.004578
iter 200: 0.004353
iter 300: 0.004136
iter 400: 0.003929
iter 500: 0.003733
iter 600: 0.003546
iter 700: 0.003369
iter 800: 0.003200
iter 900: 0.003040
iter 1000: 0.002888
test 1 result: 8.41483678608e-10
iter 100: 0.000000
iter 200: 0.000000
iter 300: 0.000000
iter 400: 0.000000
iter 500: 0.000000
iter 600: 0.000000
iter 700: 0.000000
iter 800: 0.000000
iter 900: 0.000000
iter 1000: 0.000000
test 2 result: 0.0
iter 100: 0.041205
iter 200: 0.039181
iter 300: 0.037222
iter 400: 0.035361
iter 500: 0.033593
iter 600: 0.031913
iter 700: 0.030318
iter 800: 0.028802
iter 900: 0.027362
iter 1000: 0.025994
test 3 result: -2.52445103582e-09



In [ ]:
import random
import numpy as np
from utils.treebank import StanfordSentiment
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import time

#from q3_word2vec import *
#from q3_sgd import *

# Reset the random seed to make sure that everyone gets the same results
random.seed(314)
dataset = StanfordSentiment()
tokens = dataset.tokens()
nWords = len(tokens)

# We are going to train 10-dimensional vectors for this assignment
dimVectors = 10

# Context size
C = 5

# Reset the random seed to make sure that everyone gets the same results
random.seed(31415)
np.random.seed(9265)

startTime=time.time()
wordVectors = np.concatenate(
    ((np.random.rand(nWords, dimVectors) - 0.5) /
       dimVectors, np.zeros((nWords, dimVectors))),
    axis=0)
wordVectors = sgd(
    lambda vec: word2vec_sgd_wrapper(skipgram, tokens, vec, dataset, C,
        negSamplingCostAndGradient),
    wordVectors, 0.3, 40000, None, True, PRINT_EVERY=10)
# Note that normalization is not called here. This is not a bug,
# normalizing during training loses the notion of length.

print "sanity check: cost at convergence should be around or below 10"
print "training took %d seconds" % (time.time() - startTime)

# concatenate the input and output word vectors
wordVectors = np.concatenate(
    (wordVectors[:nWords,:], wordVectors[nWords:,:]),
    axis=0)
# wordVectors = wordVectors[:nWords,:] + wordVectors[nWords:,:]

visualizeWords = [
    "the", "a", "an", ",", ".", "?", "!", "``", "''", "--",
    "good", "great", "cool", "brilliant", "wonderful", "well", "amazing",
    "worth", "sweet", "enjoyable", "boring", "bad", "waste", "dumb",
    "annoying"]

visualizeIdx = [tokens[word] for word in visualizeWords]
visualizeVecs = wordVectors[visualizeIdx, :]
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeIdx) * temp.T.dot(temp)
U,S,V = np.linalg.svd(covariance)
coord = temp.dot(U[:,0:2])

for i in xrange(len(visualizeWords)):
    plt.text(coord[i,0], coord[i,1], visualizeWords[i],
        bbox=dict(facecolor='green', alpha=0.1))

plt.xlim((np.min(coord[:,0]), np.max(coord[:,0])))
plt.ylim((np.min(coord[:,1]), np.max(coord[:,1])))

plt.savefig('q3_word_vectors.png')